In [233]:
! pip install --quiet tweepy
! pip install --quiet python-dotenv
! pip install --quiet gcsfs
! pip install --quiet orjson
! pip install --quiet clize
! pip install --quiet pymongo

In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
load_dotenv()

True

In [14]:
from rehydrate.rehydrate import *

spark = build_spark()

In [55]:
# table = 'tweets-2017-10'
month = '201610'

tweets = spark.read.parquet(f'gs://spain-tweets/ub-originals') \
                   .where(f'month = {month}')

# .where(tweets.text.rlike('^RT\s')) \

In [57]:
schema = read_schema('gs://spain-tweets/schemas/tweet-3.pickle')

In [58]:
percentage = 0.0000001

tweets = tweets \
    .sample(percentage) \
    .rdd \
    .map(lambda x: x.asDict()) \
    .mapPartitions(rehydrate, preservesPartitioning=True) \
    .map(replace_timestamps_in_dat)

    # tdf = spark.read.json(tweets.map(lambda x: orjson.dumps(x).decode('utf-8')))
    # tweet_json_with_timestamps(spark, tdf)

df = spark.createDataFrame(tweets, schema)

In [ ]:
from copy import deepcopy

def replace_with_type(schema, target, NewType):
    schema = deepcopy(schema)
    if hasattr(schema, 'fields'):
        for field in schema.fields:
            if field.name == target:
                field.dataType = NewType()
            else:
                field.dataType = replace_with_type(field.dataType, target, NewType)
        return schema
    elif hasattr(schema, 'elementType'):
        schema.elementType = replace_with_type(schema.elementType, target, NewType)
        return schema
    else:
        return schema

In [ ]:
from copy import deepcopy

def get_field(struct, name):
    return [f for f in schema.fields if f.name == name][0]

def filter_fields(struct, fields):
    struct = deepcopy(struct)
    struct.fields = [f for f in struct.fields if f.name not in fields]
    return struct


In [60]:
tweets

PythonRDD[41] at RDD at PythonRDD.scala:53

In [ ]:
schema

In [ ]:
df

In [61]:
t = tweets.collect()

In [ ]:
len(t)

In [71]:
def add_random(d):
    d['qqqq'] = { 'foo': 'bar' }
    return d


tt = tweets.map(add_random)

In [73]:
dd = spark.createDataFrame(tt, schema)

In [ ]:
dd.take(10)

In [43]:
status_fields = [f.name for f in schema.fields]
rt_status_fields = [f.name for f in get_field(schema, 'retweeted_status').dataType]
qt_status_fields = [f.name for f in get_field(schema, 'quoted_status').dataType]

for f in status_fields:
    if f not in rt_status_fields:
        print(f)

retweeted_status
th_original
th_rehydrated


In [ ]:
# is there some way to control the casting into the Tweet type, without using spark? 

In [ ]:
# check what happens when new fields are added that aren't in schema (should ignore them)

In [ ]:
# get base status
# remove retweet_status,quoted_status
# get user
# replace source_user with user


# add quoted_status = status - (retweeted and quoted)
# add retweeted status = status (as is, without retweeted)


#source_user is a user

In [ ]:
# retweeted_status is just status without retweeted_status
# quoted_status is just status without retweeted_status and quoted_status

In [ ]:
# retweeted status has quoted status
# quoted status does not have retweeted_satus
# (there's only one retweeted status)

In [33]:
lookup = {
    'source_user': 'user'
}

['entities', 'extended_entities', 'user', 'coordinates', 'place']

# user has entities (replace user later)
# place has coordinates (replace place later)

entities = get_field(schema, 'entities')
extended_entities = get_field(schema, 'extended_entities')
coordinates = get_field(schema, 'coordinates')
place = get_field(schema, 'place')

In [17]:
df.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- features: struct (nullable = true)
 |    |    |    |    |-- all: struct (nullable = true)
 |    |    |    |    |    |-- tags: array (nullable = true)
 |    |    |    |    |    |    |-- elem

In [72]:
tweets.createOrReplaceTempView('tweets')

In [ ]:
# find . -name *.sql | while read fi; do cat $fi | docker run -i --net host --rm mysql mysql -h 0.0.0.0 -D tweeter; done


In [19]:
df = spark.sql('SELECT *, existing.id as exists FROM tweets LEFT JOIN existing ON tweets.id = existing.id')

In [4]:
tweets.cache()

PythonRDD[11] at RDD at PythonRDD.scala:53

In [ ]:
tweets.count()

In [ ]:
import orjson

tdf = spark.read.json(tweets.map(lambda x: orjson.dumps(x).decode('utf-8')))


In [ ]:
tdf.printSchema()

In [18]:
tweets

PythonRDD[11] at RDD at PythonRDD.scala:53

In [ ]:
# df = tweet_json_with_timestamps(spark, tdf)
df.printSchema()

In [25]:
from gcsfs import GCSFileSystem
import pickle

fs = GCSFileSystem(project='trollhunters', token = '/home/jovyan/work/key.json')

with fs.open('spain-tweets/schemas/tweet-3.pickle', 'wb') as f: 
    pickle.dump(df.schema, f)